This project tries to classify the data from EEG signals as high Valance or low Valance.

In [2]:
import pandas as pd
import numpy as np
import pickle
import math
import statistics as stat
import scipy.stats as scStat

### The DEAP Dataset

The dataset contains 40 experiments for each of the 32 participants. The labels array contain the valence, arousal, dominance and liking ratings for each participant for each of the 40 experiments. The data array contains 8064 physiological/EEG signal data from 40 different channels for each of the 40 experiments for each of the 32 participants.

### Feature Extraction

We divide the 8064 readings per channel, into 10 batches of approximately 807 readings each. For each batch we extract the mean, median, maximum, minimum, standard deviation, variance, range, skewness and kurtosis values for the 807 readings. Hence for each of the 10 batches of a single channel we extract 9 values mentioned above, we get 90 values as our processed dataset. We further add the net mean, median, maximum, minimum, standard deviation, variance, range, skewness and kurtosis values for the entire 8064 readings along with the experiment and participant number to our dataset, bringing it up to 101 values per channel.

In [3]:
def extract_features(data, trial, participantNumber):
    extData = []
    for x in np.array_split(data, 10):
        extData.extend(calc_features(x))
    extData.extend(calc_features(data))
    extData.append(participantNumber)
    extData.append(participantNumber)
    return extData

In [4]:
def calc_features(array):
    return [stat.mean(array),
                stat.median(array),
                stat.variance(array),
                stat.stdev(array),
                max(array),
                min(array),
                scStat.mode(array)[0][0],
                scStat.kurtosis(array),
                scStat.skew(array, axis=0, bias=True)]

Features from each channel are extracted and appended to a df so that it can be stored into a csv file and accessed later.

In [5]:
def process_data_file(fileName, participantNumber):
    with open(fileName, 'rb') as f: content = pickle.load(f, encoding='latin1')
    data = content['data']
    labels = content['labels']
    extracted_features = []
    for index, trialData in enumerate(data):
        for i, channelData in enumerate(trialData):
            extracted_features.append(extract_features(channelData, index, participantNumber))
    df = pd.DataFrame(extracted_features)
    df['Valance Label'] = list(labels[:,0])*int(len(df)/len(labels))
    df['Arousal Label'] = list(labels[:,1])*int(len(df)/len(labels))
    return df

### Reading Data from DEAP Dataset

The data from DEAP Dataset .dat files are read one by one and the extracted features are appended into a csv file.

In [6]:
files = ['s05.dat']
participants = [int(x.split('.')[0][-1]) for x in files]
for f, participantNumber in zip(files, participants):
    process_data_file(f, participantNumber).to_csv('ExtractedFeatures.csv', mode='a', index=False, header=False)

FileNotFoundError: [Errno 2] No such file or directory: 's05.dat'

### Learning from extracted features

If you're reading from the provided csv file, learning starts here

In [7]:
columns = [str(i) for i in range(0,99)]
columns.extend(['experiment No', 'participant No', 'Valance Label', 'Arousal Label'])
df = pd.read_csv('ExtractedFeatures.csv', header=None, names=columns)

In [8]:
df['Valance Label'] = df['Valance Label'].apply(lambda x: 1 if x>5 else 0)
df['Arousal Label'] = df['Arousal Label'].apply(lambda x: 1 if x>5 else 0)

In [9]:
X = df[df.columns.difference(['Valance Label', 'Arousal Label'])].values

y_valance = np.array([x for i, x in enumerate(df['Valance Label'].values) if i%1640 < 40])
y_arousal = np.array([x for i, x in enumerate(df['Arousal Label'].values) if i%1640 < 40])

In [10]:
X = X.reshape(int(X.shape[0]/40),40,101)

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

### Building basic CNN Model
This model follows the setup recommended in the paper.

In [12]:
def build_model():
    model = Sequential()
    
    model.add(Conv2D(100, (3,3), padding="valid", activation='tanh', input_shape = (40, 101, 1)))
    
    model.add(Conv2D(100, (3, 3), activation='tanh' ))
    
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(128, activation='tanh'))
    model.add(Dropout(0.50))
    
    model.add(Dense(2, activation='softplus'))
    
    sgd = SGD(lr = 0.00001, momentum = 0.9, nesterov = True)
    model.compile(loss ='categorical_crossentropy', optimizer = sgd,metrics=['accuracy'])
    
    return model

###### Building a model for tuning using hyperparameters
The function sets hyperparameters for the following:
- No. of input layers into 1st Conv layer
- No. subsequent conv layers
- No. of units in these layers
- No. of Dense layers
- Learning rate

In [39]:
def build_hyperparameter_model(hp):
    model = Sequential()
    
    model.add(Conv2D(hp.Int("input_units", min_value=50, max_value=500, step=10),
                     (3,3), padding="valid", activation='tanh', input_shape = (40, 101, 1)))
    
    for i in range(hp.Int("conv_layers",1, 8)):
        model.add(Conv2D(hp.Int(f"conv_{i}_units", min_value=32, max_value=512, step=32),
                         (3, 3), activation='tanh' ))
    
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.50))
    
    model.add(Flatten())
    for i in range(hp.Int("dense_layers",0, 8)):
        model.add(Dense(hp.Int(f"dense_{i}_units", min_value=64, max_value=512, step=32), activation='tanh'))
    
    model.add(Dropout(0.50))
    
    model.add(Dense(2, activation='softplus'))
    
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]) 
    
    sgd = SGD(lr = hp_learning_rate, momentum = 0.9, nesterov = True)
    model.compile(loss ='categorical_crossentropy', optimizer = sgd,metrics=['accuracy'])
    
    return model

### One-Hot encoding labels

In [14]:
y_valance = to_categorical(y_valance, num_classes=2)
y_arousal = to_categorical(y_arousal, num_classes=2)

### KFold cross validation before hyperparameter tuning

In [41]:
def kfold_validate(X, y, model):
    model = build_model()
    kf = KFold(n_splits=int(len(X)/40), shuffle=False)
    losses = []
    accuracies = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        X_train = X_train.reshape(X_train.shape[0], 40, 101, 1)
        X_test = X_test.reshape(X_test.shape[0], 40, 101, 1)
        
        history = model.fit(X_train, y_train, batch_size = 50, 
                            epochs = 10, validation_data = (X_test, y_test))
        score = model.evaluate(X_test, y_test, batch_size = 1)
        losses.append(score[0])
        accuracies.append(score[1])
        
        return (losses, accuracies)

In [28]:
losses, accuracies = kfold_validate(X, y_valance, build_model())
print("max loss = ",max(losses))
print("max accuracy = ",max(accuracies))

Train on 200 samples, validate on 40 samples
Epoch 1/5
200/200 [==============================] - ETA: 13s - loss: 1.0210 - acc: 0.44 - ETA: 8s - loss: 0.9572 - acc: 0.4300 - ETA: 4s - loss: 0.9224 - acc: 0.480 - 19s 93ms/sample - loss: 0.8566 - acc: 0.5300 - val_loss: 0.7361 - val_acc: 0.5500
Epoch 2/5
200/200 [==============================] - ETA: 11s - loss: 0.7619 - acc: 0.54 - ETA: 7s - loss: 0.8047 - acc: 0.5000 - ETA: 3s - loss: 0.7995 - acc: 0.506 - 16s 79ms/sample - loss: 0.7735 - acc: 0.5250 - val_loss: 0.7083 - val_acc: 0.5250
Epoch 3/5
200/200 [==============================] - ETA: 14s - loss: 0.8309 - acc: 0.54 - ETA: 8s - loss: 0.7966 - acc: 0.5900 - ETA: 4s - loss: 0.8300 - acc: 0.553 - 17s 86ms/sample - loss: 0.8155 - acc: 0.5400 - val_loss: 0.7412 - val_acc: 0.4750
Epoch 4/5
200/200 [==============================] - ETA: 10s - loss: 0.7807 - acc: 0.50 - ETA: 6s - loss: 0.7903 - acc: 0.5300 - ETA: 3s - loss: 0.7524 - acc: 0.546 - 15s 73ms/sample - loss: 0.8049 - acc:

## Tuning using Keras-Tuner
- RandomSearch
- Bayesian optimisation
- Hyperband Tuning

In [24]:
import kerastuner as kt
from kerastuner.tuners import RandomSearch, BayesianOptimization, Hyperband
import time

X_train, X_test, y_train, y_test = train_test_split(X, y_valance, random_state=0)

X_train = X_train.reshape(X_train.shape[0], 40, 101, 1)
X_test = X_test.reshape(X_test.shape[0], 40, 101, 1)

LOG_DIR = f'{int(time.time())}'

rand_tuner = RandomSearch(build_hyperparameter_model, 
                    objective='acc',
                    max_trials=5,
                    executions_per_trial = 1,
                    directory=LOG_DIR)

rand_tuner.search(x=X_train,
            y=y_train,
            epochs=10,
            batch_size=50,
            validation_data=(X_test, y_test))

Train on 180 samples, validate on 60 samples
Epoch 1/10
180/180 [==============================] - ETA: 58s - loss: 0.8327 - acc: 0.60 - ETA: 35s - loss: 0.7993 - acc: 0.52 - ETA: 13s - loss: 0.7963 - acc: 0.50 - 90s 499ms/sample - loss: 0.8185 - acc: 0.4778 - val_loss: 0.7721 - val_acc: 0.4167
Epoch 2/10
180/180 [==============================] - ETA: 1:00 - loss: 0.6639 - acc: 0.600 - ETA: 36s - loss: 0.6337 - acc: 0.630 - ETA: 13s - loss: 0.6516 - acc: 0.63 - 93s 515ms/sample - loss: 0.6462 - acc: 0.6333 - val_loss: 0.7780 - val_acc: 0.4667
Epoch 3/10
180/180 [==============================] - ETA: 1:00 - loss: 0.6432 - acc: 0.660 - ETA: 37s - loss: 0.6315 - acc: 0.660 - ETA: 14s - loss: 0.6192 - acc: 0.68 - 99s 551ms/sample - loss: 0.6238 - acc: 0.6833 - val_loss: 0.7772 - val_acc: 0.5000
Epoch 4/10
180/180 [==============================] - ETA: 1:04 - loss: 0.6985 - acc: 0.500 - ETA: 38s - loss: 0.6587 - acc: 0.600 - ETA: 13s - loss: 0.6194 - acc: 0.62 - 88s 489ms/sample - loss: 

Train on 180 samples, validate on 60 samples
Epoch 1/10
180/180 [==============================] - ETA: 20s - loss: 0.8638 - acc: 0.54 - ETA: 12s - loss: 0.8815 - acc: 0.44 - ETA: 4s - loss: 0.8304 - acc: 0.4667 - 32s 177ms/sample - loss: 0.8422 - acc: 0.4444 - val_loss: 0.6828 - val_acc: 0.5667
Epoch 2/10
180/180 [==============================] - ETA: 18s - loss: 0.8295 - acc: 0.46 - ETA: 11s - loss: 0.7915 - acc: 0.46 - ETA: 4s - loss: 0.7698 - acc: 0.4933 - 30s 165ms/sample - loss: 0.7580 - acc: 0.5056 - val_loss: 0.6885 - val_acc: 0.5500
Epoch 3/10
180/180 [==============================] - ETA: 18s - loss: 0.7385 - acc: 0.52 - ETA: 11s - loss: 0.7880 - acc: 0.47 - ETA: 4s - loss: 0.7753 - acc: 0.4533 - 29s 161ms/sample - loss: 0.7583 - acc: 0.4722 - val_loss: 0.7064 - val_acc: 0.4833
Epoch 4/10
180/180 [==============================] - ETA: 19s - loss: 0.6937 - acc: 0.58 - ETA: 11s - loss: 0.7350 - acc: 0.54 - ETA: 4s - loss: 0.7521 - acc: 0.5067 - 30s 169ms/sample - loss: 0.747

Train on 180 samples, validate on 60 samples
Epoch 1/10
180/180 [==============================] - ETA: 38s - loss: 0.6768 - acc: 0.58 - ETA: 25s - loss: 0.7081 - acc: 0.52 - ETA: 9s - loss: 0.7208 - acc: 0.5000 - 70s 390ms/sample - loss: 0.7243 - acc: 0.5056 - val_loss: 0.7301 - val_acc: 0.4333
Epoch 2/10
180/180 [==============================] - ETA: 42s - loss: 0.6447 - acc: 0.54 - ETA: 26s - loss: 0.6948 - acc: 0.53 - ETA: 10s - loss: 0.7099 - acc: 0.53 - 67s 372ms/sample - loss: 0.7104 - acc: 0.5278 - val_loss: 0.7318 - val_acc: 0.4333
Epoch 3/10
180/180 [==============================] - ETA: 38s - loss: 0.7753 - acc: 0.36 - ETA: 24s - loss: 0.6893 - acc: 0.52 - ETA: 9s - loss: 0.6581 - acc: 0.6000 - 64s 355ms/sample - loss: 0.6773 - acc: 0.5778 - val_loss: 0.7335 - val_acc: 0.4500
Epoch 4/10
180/180 [==============================] - ETA: 40s - loss: 0.7549 - acc: 0.48 - ETA: 24s - loss: 0.7263 - acc: 0.53 - ETA: 8s - loss: 0.7027 - acc: 0.5667 - 61s 338ms/sample - loss: 0.6913

Train on 180 samples, validate on 60 samples
Epoch 1/10
180/180 [==============================] - ETA: 1:16 - loss: 0.7031 - acc: 0.520 - ETA: 44s - loss: 0.6988 - acc: 0.520 - ETA: 16s - loss: 0.6981 - acc: 0.48 - 114s 633ms/sample - loss: 0.6983 - acc: 0.4889 - val_loss: 0.7150 - val_acc: 0.4833
Epoch 2/10
180/180 [==============================] - ETA: 1:13 - loss: 0.6892 - acc: 0.560 - ETA: 44s - loss: 0.7035 - acc: 0.550 - ETA: 16s - loss: 0.6999 - acc: 0.54 - 114s 635ms/sample - loss: 0.7102 - acc: 0.5167 - val_loss: 0.7165 - val_acc: 0.4667
Epoch 3/10
180/180 [==============================] - ETA: 1:12 - loss: 0.6630 - acc: 0.680 - ETA: 43s - loss: 0.6959 - acc: 0.540 - ETA: 16s - loss: 0.7027 - acc: 0.50 - 106s 586ms/sample - loss: 0.7014 - acc: 0.5167 - val_loss: 0.7182 - val_acc: 0.4333
Epoch 4/10
180/180 [==============================] - ETA: 1:07 - loss: 0.6965 - acc: 0.600 - ETA: 41s - loss: 0.7083 - acc: 0.550 - ETA: 15s - loss: 0.7019 - acc: 0.55 - 107s 596ms/sample -

Train on 180 samples, validate on 60 samples
Epoch 1/10
180/180 [==============================] - ETA: 3:00 - loss: 0.7416 - acc: 0.360 - ETA: 1:49 - loss: 0.7187 - acc: 0.470 - ETA: 38s - loss: 0.7184 - acc: 0.493 - 253s 1s/sample - loss: 0.7125 - acc: 0.5056 - val_loss: 0.7570 - val_acc: 0.4500
Epoch 2/10
180/180 [==============================] - ETA: 3:09 - loss: 0.6237 - acc: 0.680 - ETA: 1:42 - loss: 0.6767 - acc: 0.600 - ETA: 39s - loss: 0.6716 - acc: 0.606 - 257s 1s/sample - loss: 0.6629 - acc: 0.6222 - val_loss: 0.7144 - val_acc: 0.4667
Epoch 3/10
180/180 [==============================] - ETA: 3:07 - loss: 0.6501 - acc: 0.560 - ETA: 1:40 - loss: 0.6700 - acc: 0.550 - ETA: 39s - loss: 0.6662 - acc: 0.566 - 252s 1s/sample - loss: 0.6504 - acc: 0.5833 - val_loss: 0.7534 - val_acc: 0.4667
Epoch 4/10
180/180 [==============================] - ETA: 2:15 - loss: 0.6257 - acc: 0.680 - ETA: 1:23 - loss: 0.5895 - acc: 0.700 - ETA: 31s - loss: 0.6274 - acc: 0.660 - 213s 1s/sample - los

INFO:tensorflow:Oracle triggered exit


In [33]:
best_model_randSearch = tuner.get_best_models(1)[0]

In [36]:
bayesian_tuner = kt.BayesianOptimization(build_hyperparameter_model,
                                         objective='val_acc', 
                                         max_trials=5,
                                         executions_per_trial = 1,
                                         directory=LOG_DIR)

bayesian_tuner.search(x=X_train,
                      y=y_train,
                      epochs=10,
                      batch_size=50,
                      validation_data=(X_test, y_test))

Train on 180 samples, validate on 60 samples
Epoch 1/10
180/180 [==============================] - ETA: 18s - loss: 0.7367 - acc: 0.46 - ETA: 9s - loss: 0.7091 - acc: 0.5600 - ETA: 3s - loss: 0.7210 - acc: 0.526 - 21s 117ms/sample - loss: 0.7213 - acc: 0.5222 - val_loss: 0.7152 - val_acc: 0.5167
Epoch 2/10
180/180 [==============================] - ETA: 12s - loss: 0.6703 - acc: 0.62 - ETA: 7s - loss: 0.6912 - acc: 0.6200 - ETA: 2s - loss: 0.6825 - acc: 0.600 - 18s 101ms/sample - loss: 0.6882 - acc: 0.6000 - val_loss: 0.7861 - val_acc: 0.4667
Epoch 3/10
180/180 [==============================] - ETA: 12s - loss: 0.6217 - acc: 0.62 - ETA: 7s - loss: 0.6063 - acc: 0.6300 - ETA: 2s - loss: 0.6015 - acc: 0.660 - 19s 106ms/sample - loss: 0.6130 - acc: 0.6611 - val_loss: 0.7149 - val_acc: 0.5167
Epoch 4/10
180/180 [==============================] - ETA: 11s - loss: 0.5956 - acc: 0.68 - ETA: 7s - loss: 0.6383 - acc: 0.6200 - ETA: 2s - loss: 0.6145 - acc: 0.660 - 18s 99ms/sample - loss: 0.5986

Train on 180 samples, validate on 60 samples
Epoch 1/10
180/180 [==============================] - ETA: 1:28 - loss: 0.7463 - acc: 0.320 - ETA: 44s - loss: 0.7466 - acc: 0.370 - ETA: 13s - loss: 0.7408 - acc: 0.40 - 111s 615ms/sample - loss: 0.7420 - acc: 0.4111 - val_loss: 0.7102 - val_acc: 0.5000
Epoch 2/10
180/180 [==============================] - ETA: 1:03 - loss: 0.7302 - acc: 0.480 - ETA: 36s - loss: 0.6963 - acc: 0.520 - ETA: 11s - loss: 0.7014 - acc: 0.49 - 99s 552ms/sample - loss: 0.6990 - acc: 0.5000 - val_loss: 0.7139 - val_acc: 0.4667
Epoch 3/10
180/180 [==============================] - ETA: 36s - loss: 0.7312 - acc: 0.50 - ETA: 21s - loss: 0.7066 - acc: 0.52 - ETA: 8s - loss: 0.7045 - acc: 0.5200 - 56s 310ms/sample - loss: 0.7149 - acc: 0.4944 - val_loss: 0.7201 - val_acc: 0.4000
Epoch 4/10
180/180 [==============================] - ETA: 54s - loss: 0.7244 - acc: 0.54 - ETA: 28s - loss: 0.7058 - acc: 0.54 - ETA: 9s - loss: 0.6942 - acc: 0.5667 - 79s 441ms/sample - loss: 

Train on 180 samples, validate on 60 samples
Epoch 1/10
180/180 [==============================] - ETA: 2:19 - loss: 0.8767 - acc: 0.340 - ETA: 1:14 - loss: 0.7841 - acc: 0.420 - ETA: 26s - loss: 0.7773 - acc: 0.466 - 173s 963ms/sample - loss: 0.7664 - acc: 0.4833 - val_loss: 0.7585 - val_acc: 0.5000
Epoch 2/10
180/180 [==============================] - ETA: 1:47 - loss: 0.7832 - acc: 0.440 - ETA: 1:04 - loss: 0.8007 - acc: 0.440 - ETA: 23s - loss: 0.7725 - acc: 0.473 - 158s 875ms/sample - loss: 0.7535 - acc: 0.5056 - val_loss: 0.7586 - val_acc: 0.5000
Epoch 3/10
180/180 [==============================] - ETA: 1:42 - loss: 0.7774 - acc: 0.460 - ETA: 1:04 - loss: 0.8211 - acc: 0.450 - ETA: 24s - loss: 0.8038 - acc: 0.453 - 161s 895ms/sample - loss: 0.7947 - acc: 0.4833 - val_loss: 0.7587 - val_acc: 0.5000
Epoch 4/10
180/180 [==============================] - ETA: 1:42 - loss: 0.7408 - acc: 0.560 - ETA: 1:04 - loss: 0.7080 - acc: 0.560 - ETA: 24s - loss: 0.7374 - acc: 0.513 - 160s 887ms/

Train on 180 samples, validate on 60 samples
Epoch 1/10
180/180 [==============================] - ETA: 1:31 - loss: 0.8728 - acc: 0.400 - ETA: 57s - loss: 0.8153 - acc: 0.490 - ETA: 21s - loss: 0.8069 - acc: 0.47 - 147s 814ms/sample - loss: 0.8183 - acc: 0.4611 - val_loss: 0.7179 - val_acc: 0.5500
Epoch 2/10
180/180 [==============================] - ETA: 2:03 - loss: 0.8467 - acc: 0.320 - ETA: 1:15 - loss: 0.7507 - acc: 0.480 - ETA: 26s - loss: 0.7975 - acc: 0.446 - 174s 966ms/sample - loss: 0.8003 - acc: 0.4500 - val_loss: 0.7147 - val_acc: 0.5500
Epoch 3/10
180/180 [==============================] - ETA: 1:42 - loss: 0.8401 - acc: 0.420 - ETA: 1:04 - loss: 0.8335 - acc: 0.400 - ETA: 24s - loss: 0.8434 - acc: 0.406 - 160s 886ms/sample - loss: 0.8501 - acc: 0.4111 - val_loss: 0.7108 - val_acc: 0.5500
Epoch 4/10
180/180 [==============================] - ETA: 1:51 - loss: 0.8148 - acc: 0.500 - ETA: 1:07 - loss: 0.8566 - acc: 0.460 - ETA: 25s - loss: 0.8528 - acc: 0.466 - 166s 922ms/sa

Train on 180 samples, validate on 60 samples
Epoch 1/10
180/180 [==============================] - ETA: 4:37 - loss: 0.7060 - acc: 0.520 - ETA: 2:41 - loss: 0.6994 - acc: 0.570 - ETA: 1:01 - loss: 0.7200 - acc: 0.526 - 402s 2s/sample - loss: 0.7105 - acc: 0.5333 - val_loss: 0.7456 - val_acc: 0.4500
Epoch 2/10
180/180 [==============================] - ETA: 4:11 - loss: 0.7073 - acc: 0.580 - ETA: 2:47 - loss: 0.6729 - acc: 0.620 - ETA: 1:04 - loss: 0.6714 - acc: 0.620 - 413s 2s/sample - loss: 0.6716 - acc: 0.6111 - val_loss: 0.7274 - val_acc: 0.4667
Epoch 3/10
180/180 [==============================] - ETA: 3:45 - loss: 0.6280 - acc: 0.660 - ETA: 2:18 - loss: 0.6490 - acc: 0.610 - ETA: 52s - loss: 0.6404 - acc: 0.613 - 353s 2s/sample - loss: 0.6384 - acc: 0.6278 - val_loss: 0.7358 - val_acc: 0.5000
Epoch 4/10
180/180 [==============================] - ETA: 5:44 - loss: 0.6657 - acc: 0.620 - ETA: 3:27 - loss: 0.6046 - acc: 0.700 - ETA: 1:15 - loss: 0.6014 - acc: 0.713 - 514s 3s/sample - 

INFO:tensorflow:Oracle triggered exit


In [37]:
best_model_bayesian = bayesian_tuner.get_best_models(1)[0]

In [38]:
best_model_bayesian.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 38, 99, 360)       3600      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 36, 97, 128)       414848    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 95, 224)       258272    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 93, 128)       258176    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 91, 448)       516544    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 89, 224)       903392    
_________________________________________________________________
flatten (Flatten)            (None, 558208)            0

In [49]:
hyperband_tuner = kt.Hyperband(build_hyperparameter_model,
                               objective='val_acc', 
                               max_epochs=5, 
                               directory=LOG_DIR)

hyperband_tuner.search(x=X_train,
                      y=y_train,
                      epochs=10,
                      batch_size=50,
                      validation_data=(X_test, y_test))

INFO:tensorflow:Reloading Oracle from existing project 1600680976\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from 1600680976\untitled_project\tuner0.json
Train on 180 samples, validate on 60 samples
Epoch 1/4
180/180 [==============================] - ETA: 1:41 - loss: 0.7548 - acc: 0.440 - ETA: 47s - loss: 0.7489 - acc: 0.490 - ETA: 15s - loss: 0.7550 - acc: 0.50 - 103s 574ms/sample - loss: 0.7595 - acc: 0.4944 - val_loss: 0.6976 - val_acc: 0.5667
Epoch 2/4
180/180 [==============================] - ETA: 43s - loss: 0.7956 - acc: 0.48 - ETA: 27s - loss: 0.8404 - acc: 0.43 - ETA: 10s - loss: 0.8208 - acc: 0.43 - 72s 397ms/sample - loss: 0.8086 - acc: 0.4444 - val_loss: 0.6977 - val_acc: 0.5833
Epoch 3/4
180/180 [==============================] - ETA: 48s - loss: 0.7080 - acc: 0.52 - ETA: 28s - loss: 0.7318 - acc: 0.48 - ETA: 10s - loss: 0.7342 - acc: 0.50 - 69s 381ms/sample - loss: 0.7288 - acc: 0.5278 - val_loss: 0.6982 - val_acc: 0.5833
Epoch 4/4
180/180 [==========

Train on 180 samples, validate on 60 samples
Epoch 5/10
180/180 [==============================] - ETA: 1:13 - loss: 0.6717 - acc: 0.520 - ETA: 39s - loss: 0.6759 - acc: 0.560 - ETA: 12s - loss: 0.7265 - acc: 0.52 - 80s 446ms/sample - loss: 0.7134 - acc: 0.5333 - val_loss: 0.7136 - val_acc: 0.5167
Epoch 6/10
180/180 [==============================] - ETA: 37s - loss: 0.6750 - acc: 0.52 - ETA: 23s - loss: 0.6919 - acc: 0.56 - ETA: 8s - loss: 0.7373 - acc: 0.5267 - 60s 335ms/sample - loss: 0.7397 - acc: 0.5222 - val_loss: 0.7146 - val_acc: 0.5000
Epoch 7/10
180/180 [==============================] - ETA: 43s - loss: 0.8014 - acc: 0.52 - ETA: 25s - loss: 0.8304 - acc: 0.46 - ETA: 9s - loss: 0.7810 - acc: 0.4867 - 60s 333ms/sample - loss: 0.7658 - acc: 0.4944 - val_loss: 0.7153 - val_acc: 0.4667
Epoch 8/10
180/180 [==============================] - ETA: 36s - loss: 0.8495 - acc: 0.40 - ETA: 22s - loss: 0.8257 - acc: 0.40 - ETA: 8s - loss: 0.7914 - acc: 0.4600 - 57s 314ms/sample - loss: 0.8

Train on 180 samples, validate on 60 samples
Epoch 5/10
180/180 [==============================] - ETA: 2:35 - loss: 0.7798 - acc: 0.540 - ETA: 1:33 - loss: 0.7718 - acc: 0.540 - ETA: 34s - loss: 0.7883 - acc: 0.526 - 230s 1s/sample - loss: 0.7808 - acc: 0.5167 - val_loss: 0.7059 - val_acc: 0.5167
Epoch 6/10
180/180 [==============================] - ETA: 2:28 - loss: 0.8029 - acc: 0.460 - ETA: 1:31 - loss: 0.7912 - acc: 0.460 - ETA: 34s - loss: 0.7823 - acc: 0.493 - 228s 1s/sample - loss: 0.7936 - acc: 0.4889 - val_loss: 0.7066 - val_acc: 0.5000
Epoch 7/10
180/180 [==============================] - ETA: 2:29 - loss: 0.6817 - acc: 0.580 - ETA: 1:31 - loss: 0.6792 - acc: 0.590 - ETA: 34s - loss: 0.7202 - acc: 0.566 - 226s 1s/sample - loss: 0.7511 - acc: 0.5333 - val_loss: 0.7072 - val_acc: 0.5000
Epoch 8/10
180/180 [==============================] - ETA: 2:29 - loss: 0.6407 - acc: 0.600 - ETA: 1:31 - loss: 0.7376 - acc: 0.570 - ETA: 34s - loss: 0.7299 - acc: 0.580 - 225s 1s/sample - los

Train on 180 samples, validate on 60 samples
Epoch 1/10
180/180 [==============================] - ETA: 1:03 - loss: 0.8505 - acc: 0.460 - ETA: 38s - loss: 1.1228 - acc: 0.470 - ETA: 14s - loss: 1.0138 - acc: 0.48 - 97s 538ms/sample - loss: 0.9565 - acc: 0.4944 - val_loss: 0.7049 - val_acc: 0.4500
Epoch 2/10
180/180 [==============================] - ETA: 1:04 - loss: 0.8799 - acc: 0.380 - ETA: 39s - loss: 0.7991 - acc: 0.470 - ETA: 14s - loss: 0.7869 - acc: 0.47 - 96s 533ms/sample - loss: 0.7713 - acc: 0.4944 - val_loss: 0.7309 - val_acc: 0.4500
Epoch 3/10
180/180 [==============================] - ETA: 1:02 - loss: 0.6796 - acc: 0.640 - ETA: 38s - loss: 0.6866 - acc: 0.620 - ETA: 14s - loss: 0.7007 - acc: 0.60 - 95s 529ms/sample - loss: 0.7282 - acc: 0.5667 - val_loss: 0.7297 - val_acc: 0.4500
Epoch 4/10
180/180 [==============================] - ETA: 1:01 - loss: 0.6870 - acc: 0.600 - ETA: 38s - loss: 0.6939 - acc: 0.580 - ETA: 14s - loss: 0.6857 - acc: 0.58 - 96s 531ms/sample - los

Train on 180 samples, validate on 60 samples
Epoch 1/10
180/180 [==============================] - ETA: 18s - loss: 0.6786 - acc: 0.58 - ETA: 10s - loss: 0.7481 - acc: 0.57 - ETA: 4s - loss: 0.7239 - acc: 0.5800 - 28s 157ms/sample - loss: 0.7336 - acc: 0.5722 - val_loss: 0.6931 - val_acc: 0.4833
Epoch 2/10
180/180 [==============================] - ETA: 17s - loss: 0.7257 - acc: 0.52 - ETA: 10s - loss: 0.7329 - acc: 0.51 - ETA: 4s - loss: 0.7302 - acc: 0.5267 - 27s 147ms/sample - loss: 0.7240 - acc: 0.5278 - val_loss: 0.7095 - val_acc: 0.4333
Epoch 3/10
180/180 [==============================] - ETA: 16s - loss: 0.6581 - acc: 0.60 - ETA: 10s - loss: 0.6718 - acc: 0.56 - ETA: 3s - loss: 0.6602 - acc: 0.5800 - 26s 144ms/sample - loss: 0.6611 - acc: 0.5833 - val_loss: 0.7429 - val_acc: 0.4500
Epoch 4/10
180/180 [==============================] - ETA: 17s - loss: 0.7081 - acc: 0.58 - ETA: 10s - loss: 0.6570 - acc: 0.63 - ETA: 4s - loss: 0.6437 - acc: 0.6400 - 26s 147ms/sample - loss: 0.657

Train on 180 samples, validate on 60 samples
Epoch 1/10
180/180 [==============================] - ETA: 1:13 - loss: 0.8158 - acc: 0.460 - ETA: 44s - loss: 0.8120 - acc: 0.490 - ETA: 16s - loss: 0.8408 - acc: 0.46 - 112s 621ms/sample - loss: 0.8078 - acc: 0.5000 - val_loss: 0.9104 - val_acc: 0.4500
Epoch 2/10
180/180 [==============================] - ETA: 1:12 - loss: 0.7622 - acc: 0.580 - ETA: 44s - loss: 0.6905 - acc: 0.620 - ETA: 16s - loss: 0.7187 - acc: 0.58 - 111s 616ms/sample - loss: 0.7034 - acc: 0.5889 - val_loss: 0.7133 - val_acc: 0.4667
Epoch 3/10
180/180 [==============================] - ETA: 1:11 - loss: 0.6487 - acc: 0.580 - ETA: 44s - loss: 0.6372 - acc: 0.630 - ETA: 16s - loss: 0.6466 - acc: 0.61 - 111s 618ms/sample - loss: 0.6624 - acc: 0.5889 - val_loss: 0.7112 - val_acc: 0.5167
Epoch 4/10
180/180 [==============================] - ETA: 1:12 - loss: 0.6916 - acc: 0.540 - ETA: 44s - loss: 0.6890 - acc: 0.510 - ETA: 16s - loss: 0.6815 - acc: 0.52 - 110s 610ms/sample -

Train on 180 samples, validate on 60 samples
Epoch 1/10
180/180 [==============================] - ETA: 3:24 - loss: 0.6876 - acc: 0.600 - ETA: 2:05 - loss: 0.6931 - acc: 0.590 - ETA: 46s - loss: 0.7167 - acc: 0.520 - 312s 2s/sample - loss: 0.7078 - acc: 0.5333 - val_loss: 0.6845 - val_acc: 0.5167
Epoch 2/10
180/180 [==============================] - ETA: 3:23 - loss: 0.6501 - acc: 0.680 - ETA: 2:06 - loss: 0.7017 - acc: 0.520 - ETA: 48s - loss: 0.7177 - acc: 0.486 - 321s 2s/sample - loss: 0.7111 - acc: 0.4889 - val_loss: 0.6845 - val_acc: 0.5167
Epoch 3/10
180/180 [==============================] - ETA: 3:23 - loss: 0.7015 - acc: 0.500 - ETA: 2:08 - loss: 0.6960 - acc: 0.510 - ETA: 47s - loss: 0.6840 - acc: 0.540 - 314s 2s/sample - loss: 0.7101 - acc: 0.5056 - val_loss: 0.6845 - val_acc: 0.5167
Epoch 4/10
180/180 [==============================] - ETA: 3:23 - loss: 0.7184 - acc: 0.520 - ETA: 2:05 - loss: 0.7196 - acc: 0.530 - ETA: 47s - loss: 0.7229 - acc: 0.500 - 312s 2s/sample - los

INFO:tensorflow:Oracle triggered exit


In [50]:
best_model_hyperband = hyperband_tuner.get_best_models(1)[0]
best_model_hyperband.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 38, 99, 310)       3100      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 36, 97, 448)       1250368   
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 95, 288)       1161504   
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 17, 47, 288)       0         
_________________________________________________________________
dropout (Dropout)            (None, 17, 47, 288)       0         
_________________________________________________________________
flatten (Flatten)            (None, 230112)            0         
_________________________________________________________________
dense (Dense)                (None, 224)               5

In [44]:
losses, accuracies = kfold_validate(X, y_valance, best_model_randSearch)
print("mean loss = ",np.mean(losses))
print("mean accuracy = ",np.mean(accuracies))

mean loss =  0.7141988947987556
mean accuracy =  0.35


In [45]:
losses, accuracies = kfold_validate(X, y_valance, best_model_bayesian)
print("mean loss = ",np.mean(losses))
print("mean accuracy = ",np.mean(accuracies))

Train on 200 samples, validate on 40 samples
Epoch 1/10
200/200 [==============================] - ETA: 14s - loss: 0.7233 - acc: 0.62 - ETA: 8s - loss: 0.7602 - acc: 0.5900 - ETA: 4s - loss: 0.7852 - acc: 0.533 - 19s 96ms/sample - loss: 0.8026 - acc: 0.5100 - val_loss: 0.7463 - val_acc: 0.4750
Epoch 2/10
200/200 [==============================] - ETA: 12s - loss: 0.8681 - acc: 0.44 - ETA: 9s - loss: 0.8884 - acc: 0.4800 - ETA: 4s - loss: 0.8549 - acc: 0.506 - 20s 98ms/sample - loss: 0.8326 - acc: 0.5150 - val_loss: 0.7248 - val_acc: 0.4750
Epoch 3/10
200/200 [==============================] - ETA: 12s - loss: 0.7832 - acc: 0.44 - ETA: 8s - loss: 0.7870 - acc: 0.4500 - ETA: 4s - loss: 0.7922 - acc: 0.466 - 18s 89ms/sample - loss: 0.7941 - acc: 0.4650 - val_loss: 0.7171 - val_acc: 0.4500
Epoch 4/10
200/200 [==============================] - ETA: 12s - loss: 0.6955 - acc: 0.60 - ETA: 8s - loss: 0.7466 - acc: 0.5400 - ETA: 4s - loss: 0.7622 - acc: 0.506 - 18s 91ms/sample - loss: 0.7712 - 

In [51]:
losses, accuracies = kfold_validate(X, y_valance, best_model_hyperband)
print("mean loss = ",np.mean(losses))
print("mean accuracy = ",np.mean(accuracies))

Train on 200 samples, validate on 40 samples
Epoch 1/10
200/200 [==============================] - ETA: 15s - loss: 0.8594 - acc: 0.52 - ETA: 9s - loss: 0.8643 - acc: 0.5000 - ETA: 4s - loss: 0.8300 - acc: 0.513 - 19s 95ms/sample - loss: 0.8154 - acc: 0.5400 - val_loss: 0.7185 - val_acc: 0.5250
Epoch 2/10
200/200 [==============================] - ETA: 12s - loss: 0.8297 - acc: 0.58 - ETA: 8s - loss: 0.8771 - acc: 0.5400 - ETA: 4s - loss: 0.8931 - acc: 0.513 - 18s 89ms/sample - loss: 0.8336 - acc: 0.5450 - val_loss: 0.7556 - val_acc: 0.4500
Epoch 3/10
200/200 [==============================] - ETA: 12s - loss: 0.7160 - acc: 0.62 - ETA: 8s - loss: 0.7741 - acc: 0.6000 - ETA: 4s - loss: 0.7655 - acc: 0.593 - 18s 91ms/sample - loss: 0.7850 - acc: 0.5850 - val_loss: 0.7473 - val_acc: 0.4500
Epoch 4/10
200/200 [==============================] - ETA: 16s - loss: 0.7966 - acc: 0.48 - ETA: 9s - loss: 0.8645 - acc: 0.4500 - ETA: 4s - loss: 0.8564 - acc: 0.466 - 19s 96ms/sample - loss: 0.8192 - 